## Preprocess video  
### Download video and split it into frames  
- NOTE: SAM2 assumee that the video is stored as a list of JPEG frames with filenames like <frame_index>.jpg. Let's start by downloading a sample video, splitting it into frames, and saving them to disk. Feel free to replace SOURCE_VIDEO with the path to your video file.
- 비디오를 다운로드하여 프레임으로 나누고 디스크에 저장   

## Setup 및 라이브러리 설치  

In [ ]:
# NOTE: To make it easier for us to manage datasets, images and models we create a HOME constant.
import os

HOME = os.getcwd()
print("HOME:", HOME)

HOME: /content


In [ ]:
# Install SAM2 and dependencies
!git clone https://github.com/facebookresearch/segment-anything-2.git
%cd {HOME}/segment-anything-2
!pip install -e . -q
!python setup.py build_ext --inplace

# 필수 라이브러리 설치
!pip install -q supervision[assets] jupyter_bbox_widget
!pip install ultralytics

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 13.87 MiB/s, done.
Resolving deltas: 100% (381/381), done.
/content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52

## SAM2 체크포인트 다운로드 및 데이터 다운로드  

In [ ]:
# Download SAM2 checkpoints
!mkdir -p {HOME}/checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P {HOME}/checkpoints

# yolo11s best model
!gdown 1A6_iThtWBlzgvGqOP9QXuDYGFL809jYL

## 라이브러리 Imports 및 모델 초기화  

In [ ]:
import cv2
import torch
import base64
import numpy as np
import supervision as sv

from pathlib import Path
from supervision.assets import download_assets, VideoAssets
from sam2.build_sam import build_sam2_video_predictor

import shutil
from google.colab import files

IS_COLAB = True

if IS_COLAB:
    from google.colab import output
    output.enable_custom_widget_manager()

from jupyter_bbox_widget import BBoxWidget

In [ ]:
# NOTE: This code enables mixed-precision computing for faster deep learning. It uses bfloat16 for most calculations and, on newer NVIDIA GPUs, leverages TensorFloat-32 (TF32) for certain operations to further boost performance.
torch.autocast(device_type="cuda", dtype=torch.bfloat16).__enter__()

if torch.cuda.get_device_properties(0).major >= 8:
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CHECKPOINT = f"{HOME}/checkpoints/sam2_hiera_large.pt"
CONFIG = "sam2_hiera_l.yaml"

# sam2 model 초기화
sam2_model = build_sam2_video_predictor(CONFIG, CHECKPOINT)

## 비디오 정보 확인(fps, 프레임 수 등)

In [ ]:
video_path = "/content/test video 07.mp4" # 사용할 비디오 경로

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 원본 총 프레임 개수
duration = frame_count / fps  # 계산된 비디오 길이
cap.release()

print(f"비디오 FPS: {fps:.6f}")
print(f"비디오 총 프레임 개수: {frame_count}")
print(f"계산된 비디오 길이: {duration:.2f}초")

ZeroDivisionError: float division by zero

## 비디오 -> 프레임 분할 및 저장  
- sv.get_video_frames_generator(SOURCE_VIDEO, start=0, end=None) 함수를 사용  
- 1초에 몇 개의 프레임이 저장되었는지 확인 가능  

In [ ]:
SCALE_FACTOR = 0.5  # 이미지 크기 50% 축소
SOURCE_VIDEO = "/content/test video 07.mp4"
SOURCE_FRAMES = Path(HOME) / Path(SOURCE_VIDEO).stem
SOURCE_FRAMES.mkdir(parents=True, exist_ok=True)

frames_generator = sv.get_video_frames_generator(SOURCE_VIDEO, start=0, end=None)

images_sink = sv.ImageSink(
    target_dir_path=SOURCE_FRAMES.as_posix(),
    overwrite=True,
    image_name_pattern="{:05d}.jpg"
)

with images_sink:
    for frame in frames_generator:
        frame = sv.scale_image(frame, SCALE_FACTOR)
        images_sink.save_image(frame)

# 첫번째 프레임 및 결과 및 저장 경로
SOURCE_IMAGE = SOURCE_FRAMES / "00000.jpg"
TARGET_VIDEO = Path(HOME) / f"{Path(SOURCE_VIDEO).stem}-result-07.mp4"

In [ ]:
# 프레임 압축
folder_path = "/content/test video 07"  # 저장된 프레임이 있는 폴더
zip_path = "/content/test_video_frames_07.zip"  # 압축 파일명

shutil.make_archive(zip_path.replace(".zip", ""), 'zip', folder_path)
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Initialize the inference state
- NOTE: SAM 2 requires stateful inference for interactive video segmentation, so we need to initialize an inference state on this video. During initialization, it loads all the JPEG frames in video_path and stores their pixels in inference_state (as shown in the progress bar below).  
- SAM 2는 비디오 인터랙티브 세그멘테이션을 위해 상태를 유지하는(stateful) 추론 방식을 사용하기 때문에, 비디오에 대한 추론을 실행하기 전에 먼저 "추론 상태(inference state)"를 초기화해야 합니다.
- 초기화하는 동안, video_path에 있는 모든 JPEG 프레임을 불러와서 프레임의 픽셀 데이터를 inference_state에 저장합니다.  

## SAM2 모델 초기화 (inference 대비용)  

- NOTE: 만약 inference_state를 사용하여 이전에 **트래킹(Tracking, 추적)**을 실행했다면, 먼저 reset_state()를 호출하여 초기화해야 합니다.

In [ ]:
# sam2 모델 초기화
inference_state = sam2_model.init_state(video_path=SOURCE_FRAMES.as_posix())
sam2_model.reset_state(inference_state)

frame loading (JPEG): 100%|██████████| 379/379 [00:11<00:00, 34.21it/s]
